<a href="https://colab.research.google.com/github/Sandhiya-8/CI-CD/blob/main/Podcast_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
import re #for immplementing regular expressions
from sklearn.feature_extraction.text import TfidfVectorizer #for implementing Tf-Idf method of Text Processing
from sklearn.metrics.pairwise import linear_kernel #for getting a similarity matrix by comparing the similarity between the data-pooints

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [7]:
podcasts=pd.read_csv("podcasts.csv")
print(podcasts.shape)
podcasts.head()

(71753, 9)


,uuid,title,image,description,language,categories,website,author,itunes_id
0,8d62d3880db2425b890b986e58aca393,"Ecommerce Conversations, by Practical Ecommerce",http://is4.mzstatic.com/image/thumb/Music6/v4/...,Listen in as the Practical Ecommerce editorial...,English,Technology,http://www.practicalecommerce.com,Practical Ecommerce,8.744574e+08
1,cbbefd691915468c90f87ab2f00473f9,Eat Sleep Code Podcast,http://is4.mzstatic.com/image/thumb/Music71/v4...,On the show we’ll be talking to passionate peo...,English,Tech News | Technology,http://developer.telerik.com/,Telerik,1.015556e+09
2,73626ad1edb74dbb8112cd159bda86cf,SoundtrackAlley,http://is5.mzstatic.com/image/thumb/Music71/v4...,A podcast about soundtracks and movies from my...,English,Podcasting | Technology,https://soundtrackalley.podbean.com,Randy Andrews,1.158189e+09
3,0f50631ebad24cedb2fee80950f37a1a,The Tech M&A Podcast,http://is1.mzstatic.com/image/thumb/Music71/v4...,The Tech M&A Podcast pulls from the best of th...,English,Business News | Technology | Tech News | Business,http://www.corumgroup.com,Timothy Goddard,5.381600e+08
4,69580e7b419045839ca07af06cf0d653,"The Tech Informist - For fans of Apple, Google...",http://is4.mzstatic.com/image/thumb/Music62/v4...,The tech news show with two guys shooting the ...,English,Gadgets | Tech News | Technology,http://techinformist.com,The Tech Informist,9.160805e+08


In [8]:
podcasts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71753 entries, 0 to 71752
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   uuid         71753 non-null  object 
 1   title        71751 non-null  object 
 2   image        71753 non-null  object 
 3   description  70951 non-null  object 
 4   language     71753 non-null  object 
 5   categories   71753 non-null  object 
 6   website      71054 non-null  object 
 7   author       70253 non-null  object 
 8   itunes_id    71752 non-null  float64
dtypes: float64(1), object(8)
memory usage: 4.9+ MB


In [9]:
podcasts.language.value_counts()

language
English       58920
German         2581
French         2318
Spanish        2172
Portuguese     1056
Swedish         961
Chinese         766
Japanese        628
Italian         479
Russian         341
Dutch           301
Norwegian       242
Danish          188
Korean          114
Finnish          97
Polish           90
Estonian         81
Turkish          64
Czech            60
Catalan          56
Hungarian        46
Slovenian        40
Romanian         27
Galician         21
Ukranian         18
Swahili          12
Slovak           11
Indonesian       10
Icelandic         8
Afrikaans         8
Basque            8
Greek             7
Bulgarian         5
Serbian           5
Irish             3
Croatian          3
Albanian          2
Abkhazian         2
Afar              1
Macedonian        1
Name: count, dtype: int64

In [10]:
podcasts=podcasts[podcasts.language == "English"]
podcasts.shape

(58920, 9)

In [11]:
podcasts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58920 entries, 0 to 71752
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   uuid         58920 non-null  object 
 1   title        58919 non-null  object 
 2   image        58920 non-null  object 
 3   description  58226 non-null  object 
 4   language     58920 non-null  object 
 5   categories   58920 non-null  object 
 6   website      58267 non-null  object 
 7   author       57610 non-null  object 
 8   itunes_id    58919 non-null  float64
dtypes: float64(1), object(8)
memory usage: 4.5+ MB


In [12]:
podcasts=podcasts.dropna(subset=["description"])
podcasts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58226 entries, 0 to 71752
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   uuid         58226 non-null  object 
 1   title        58225 non-null  object 
 2   image        58226 non-null  object 
 3   description  58226 non-null  object 
 4   language     58226 non-null  object 
 5   categories   58226 non-null  object 
 6   website      57589 non-null  object 
 7   author       56975 non-null  object 
 8   itunes_id    58225 non-null  float64
dtypes: float64(1), object(8)
memory usage: 4.4+ MB


In [13]:
podcasts=podcasts.drop_duplicates("itunes_id")
podcasts.shape

(58226, 9)

In [14]:
podcasts["description_#words"]=[len(x.description.split()) for _,x in podcasts.iterrows()]
podcasts["description_#words"].describe()

count    58226.000000
mean        38.836997
std         49.879003
min          0.000000
25%         11.000000
50%         26.000000
75%         51.000000
max       5097.000000
Name: description_#words, dtype: float64

In [15]:
podcasts = podcasts[podcasts["description_#words"] > 19]
podcasts.shape

(35011, 10)

In [16]:
fav_podcasts = ["Earth 919: A Comic Book Podcast", "Digital India" ,"Top 5 Comics Podcast"]
fav_df=podcasts[podcasts.title.isin(fav_podcasts)]
fav_df

,uuid,title,image,description,language,categories,website,author,itunes_id,description_#words
9,7bbb6386cacb45518f2e5b2427c4cbb1,Digital India,http://is4.mzstatic.com/image/thumb/Music60/v4...,"Suresh Babu, founder and head of the Web Marke...",English,Podcasting | Management & Marketing | Tech New...,https://firpodcastnetwork.com/digital-india/,Digital India,1.003892e+09,29
29,600a324915904dcc8e2c8f5cf89c91be,Top 5 Comics Podcast,http://is3.mzstatic.com/image/thumb/Music128/v...,Top 5 Comics Podcast is a round table comic bo...,English,Arts | Literature,http://top5comics.podomatic.com,Top 5 Comics Podcast,5.993989e+08,36
93,a3da910ffb124b1db96bce503beda448,Earth 919: A Comic Book Podcast,http://is1.mzstatic.com/image/thumb/Music82/v4...,Earth 919 is your go-to comics podcast for #1 ...,English,Hobbies | Games & Hobbies,http://earth919comics.blogspot.com/,Mat,1.117727e+09,36


In [17]:
podcasts=podcasts[~podcasts.isin(fav_df)].sample(25000)
podcasts_df=pd.concat([fav_df,podcasts],sort=True).reset_index(drop=True)
podcasts_df=podcasts_df.dropna(subset=["description"])
print(podcasts_df.shape)
podcasts_df.head()

(25001, 10)


,author,categories,description,description_#words,image,itunes_id,language,title,uuid,website
0,Digital India,Podcasting | Management & Marketing | Tech New...,"Suresh Babu, founder and head of the Web Marke...",29.0,http://is4.mzstatic.com/image/thumb/Music60/v4...,1.003892e+09,English,Digital India,7bbb6386cacb45518f2e5b2427c4cbb1,https://firpodcastnetwork.com/digital-india/
1,Top 5 Comics Podcast,Arts | Literature,Top 5 Comics Podcast is a round table comic bo...,36.0,http://is3.mzstatic.com/image/thumb/Music128/v...,5.993989e+08,English,Top 5 Comics Podcast,600a324915904dcc8e2c8f5cf89c91be,http://top5comics.podomatic.com
2,Mat,Hobbies | Games & Hobbies,Earth 919 is your go-to comics podcast for #1 ...,36.0,http://is1.mzstatic.com/image/thumb/Music82/v4...,1.117727e+09,English,Earth 919: A Comic Book Podcast,a3da910ffb124b1db96bce503beda448,http://earth919comics.blogspot.com/
3,The Spontaneity Shop,Comedy,Ever felt like you should be better at feminis...,93.0,http://is5.mzstatic.com/image/thumb/Music122/v...,1.068941e+09,English,The Guilty Feminist,6c7cce79239f4259807d71057fd13bcf,http://guiltyfeminist.libsyn.com/podcast
4,Katy Patton,Society & Culture,Katy will have conversations with her guests a...,43.0,http://is1.mzstatic.com/image/thumb/Music71/v4...,1.180585e+09,English,Happy Playces podcast,ca5c95665f144fda85669d6fc52a8d2f,http://happyplayces.libsyn.com/podcast


In [18]:
podcasts_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25001 entries, 0 to 25002
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   author              24701 non-null  object 
 1   categories          25001 non-null  object 
 2   description         25001 non-null  object 
 3   description_#words  25001 non-null  float64
 4   image               25001 non-null  object 
 5   itunes_id           25001 non-null  float64
 6   language            25001 non-null  object 
 7   title               25001 non-null  object 
 8   uuid                25001 non-null  object 
 9   website             24739 non-null  object 
dtypes: float64(2), object(8)
memory usage: 2.1+ MB


In [19]:
tfidf_model = TfidfVectorizer(analyzer='word',ngram_range=(1,3),stop_words='english')
tf_idf = tfidf_model.fit_transform(podcasts_df['description'])
tf_idf

<25001x1354008 sparse matrix of type '<class 'numpy.float64'>'
	with 2273105 stored elements in Compressed Sparse Row format>

In [20]:
similarity_matrix = linear_kernel(tf_idf,tf_idf)
similarity_matrix

array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.00000000e+00, 6.26919207e-02, ...,
        7.78102426e-04, 6.51676183e-04, 6.89421481e-03],
       [0.00000000e+00, 6.26919207e-02, 1.00000000e+00, ...,
        2.59671628e-03, 6.48880309e-04, 2.31913324e-03],
       ...,
       [0.00000000e+00, 7.78102426e-04, 2.59671628e-03, ...,
        1.00000000e+00, 2.57002967e-03, 2.51682335e-03],
       [0.00000000e+00, 6.51676183e-04, 6.48880309e-04, ...,
        2.57002967e-03, 1.00000000e+00, 6.28916268e-04],
       [0.00000000e+00, 6.89421481e-03, 2.31913324e-03, ...,
        2.51682335e-03, 6.28916268e-04, 1.00000000e+00]])

fav1=podcasts_df[podcasts_df.title == "Top 5 Comics Podcast"].index[0]
index_recom = similarity_matrix[fav1].argsort(axis=0)[-5:-1]

for i in index_recom:
    print("Score:",similarity_matrix[fav1][i],"\t Title:",podcasts_df.title[i])
    print(podcasts_df.description[i],"\n")
print("Original Description : ",podcasts_df.description[fav1])

In [22]:
fav2=podcasts_df[podcasts_df.title == "Digital India"].index[0]
index_recom = similarity_matrix[fav2].argsort(axis=0)[-5:-1]

for i in index_recom:
    print("Score:",similarity_matrix[fav2][i],"\t Title:",podcasts_df.title[i])
    print(podcasts_df.description[i],"\n")
print("Original Description : ",podcasts_df.description[fav2])

Score: 0.06051188633473597 	 Title: All Day Ruckoff
Brian, from All Day Ruckoff, talks about rucking, gear, fitness, and beer with those in the community. After completing GORUCK Challenge Class 053 Brian started the website All Day Ruckoff to encourage and provide insight to others who wanted to take on the GORUCK Challenge. Years after the formation of All Day Ruckoff the All Day Ruckoff Podcast is born. Listen to interviews with athletes, event organizers, cadre, ruck clubs, and those who make this community special. 

Score: 0.06263071352209215 	 Title: Everyday Creative People
A weekly interview series with a diverse selection of doers, dreamers, and makers who are cultivating a creative practice in their lives, to talk about the good, the bad, and the ugly of creative living. For anyone who wishes they had more time and freedom to play, who struggles with creative blocks, or who is trying to figure out how to make a living while making art, we're here to stumble through the madne

In [23]:
fav3=podcasts_df[podcasts_df.title == "Earth 919: A Comic Book Podcast"].index[0]
index_recom = similarity_matrix[fav3].argsort(axis=0)[-5:-1]

for i in index_recom:
    print("Score:",similarity_matrix[fav3][i],"\t Title:",podcasts_df.title[i])
    print(podcasts_df.description[i],"\n")
print("Original Description : ",podcasts_df.description[fav3])

Score: 0.07161993188949836 	 Title: Prequel Sequel Requel: Podcast Version
The audio version of Prequel Sequel Requel, the video series where films are given a prequel, sequel and a reboot or remake by some chaps with more time on their hands than most other people. Watch the video version on YouTube and carry the audio around on your person. 

Score: 0.0745436857161243 	 Title: Clarion Podcasts
Bringing you vital facts and perspectives on Islamist extremism and how to combat it directly from leading activists in the field. 

Clarion Podcast is brought to you by Clarion Project, dedicated to challenging Islamist extremism and promoting dialogue with Muslim human rights activists. 

Score: 0.08017529123084909 	 Title: Uncharted Podcast
Join explorers from around the world in discovering all things "uncharted." Find new places, sports, activities, people and other marvels both far away and closer to home. 

Score: 0.0859614995361255 	 Title: The Greenhouse Culture
The Greenhouse Culture 

In [24]:
fav4=podcasts_df[podcasts_df.title == "Eat Sleep Code Podcast"].index[0]
index_recom = similarity_matrix[fav4].argsort(axis=0)[-5:-1]

for i in index_recom:
    print("Score:",similarity_matrix[fav4][i],"\t Title:",podcasts_df.title[i])
    print(podcasts_df.description[i],"\n")
print("Original Description : ",podcasts_df.description[fav4])

Score: 0.03743472439674739 	 Title: Let’s Talk MMA
Everything MMA related from the fighters to the sport to the money to the action all of it right here every day live‼️‼️‼️ 

Score: 0.039015324184082184 	 Title: Last Men On Earth
It’s 2015. The male of the species has been cowered to near extinction by a combination of Soul Cycle feminism and excessive pass interference calls in football. From their unmarked storage closet location, Lex Jurgen and Matt Ralston broadcast the last bastion of masculine thought to a dispersed audience of survivors. Like a giant dick beacon, if that’s your thing. 

Score: 0.03976295633975173 	 Title: Matthew (2013)
True stories of the lives of famous people quickly move to the top of the best sellers' charts. Listen to this series about the engrossing story of the life of Christ as told by a once-hated tax collector, Matthew. See how his emphasis is on Christ as the promised Messianic king. Find out about the not-to-be-missed practical ways this gospel can